In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball
from skimage import color

from skimage import morphology

from sklearn import preprocessing

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
# from radio_beam.commonbeam import getMinVolEllipse

from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate


In [ ]:
def preprocess_image(image):
    # комбинация медианного фильтра, биноризации и градиента
    # у зерен значение пикселя - 0, у регионов связ. в-ва - 127,а у их границы - 254

    # image = util.img_as_ubyte(image)
    # if len(image.shape) < 3:
    #     image = image[..., np.newaxis]

    if len(image.shape)==3:
        image = color.rgb2gray(image)

    image = filters.rank.median(image, morphology.disk(3))

    global_thresh = filters.threshold_otsu(image)
    image = image > global_thresh
    binary = image*255

    grad = abs(filters.rank.gradient(binary, morphology.disk(1)))
    bin_grad = (1 - binary + grad) * 127
    bin_grad = bin_grad.astype(np.uint8)

    return bin_grad

img1 = io.imread('../ml/diffusion/data/o_bc_left_4x_768_360_768_median_generated/Ultra_Co11/0.png')
img2 = io.imread('../datasets/original/o_bc_left_4x_768/Ultra_Co11/Ultra_Co11-1_part_1.jpeg')

img1=preprocess_image(img1)
img2=preprocess_image(img2)

In [ ]:
plt.imshow(img1)

In [ ]:
def draw_edges(image, cnts, color=(0, 139, 139), r=4, e_width=5, l_width=4):
    """
    :param image: ndarray (width, height, channels)
    :param cnts: ndarray (n_cnts,n,2)
    :param color: tuple (3,)
    :return: ndarray (width, height, channels)
    """
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    #
    new_image = copy.copy(image)

    im = Image.fromarray(new_image)
    draw = ImageDraw.Draw(im)

    for j, cnt in enumerate(cnts):
        if len(cnt) > 1:
            point = cnt[0]
            x1, y1 = point[1], point[0]

            for i, point2 in enumerate(cnt):
                p2 = point2

                x2, y2 = p2[1], p2[0]

                draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=e_width)
                draw.line((y1, x1, y2, x2), fill=(255, 140, 0), width=l_width)
                x1, y1 = x2, y2

        else:
            continue

    img = np.array(im)

    return img

In [ ]:
image=img1

cnt_raw=grainMark.get_row_contours(image=image)
cnt=grainMark.get_contours(image=image)

cnt_img=draw_edges(color.gray2rgb(image),cnts=cnt, r=2, e_width=1, l_width=1)

plt.figure(figsize=(10,10))
plt.imshow(cnt_img[:256,:256])
plt.show()

angles = grainMark.get_angles(image)

plt.hist(angles,bins=20)
plt.show()

In [ ]:
image=img2

cnt_raw=grainMark.get_row_contours(image=image)
cnt=grainMark.get_contours(image=image)

cnt_img=draw_edges(color.gray2rgb(image),cnts=cnt, r=2, e_width=1, l_width=1)

plt.figure(figsize=(5,5))
plt.imshow(cnt_img[:256,:256])
plt.show()

io.imread(image,'test.pnd')

angles = grainMark.get_angles(image,tol=3)

plt.hist(angles,bins=20)
plt.show()